# Optimizing Strategy Parameters

<table style="width:100%; height:90%">
      <tr>
    <th>Parametrize the Strategy</th>
    <th>Optimizing Limits' Parameters</th>
  </tr>
  <tr>
    <td><img src="src/07_Code_Regression Strategy Limits X.png" alt="Parametrize the Strategy" style="width:100%"></td>
    <td><img src="src/07_Table_Optimize BG Default Defaults.png" alt="Optimizing Limits' Parameters" style="width:100%"></td>
  </tr>
</table>

## Load the model

In [1]:
import pickle

with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [2]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800,1.549143,UP
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.321692,UP
2016-12-12,56.803028,57.244073,56.711145,57.124622,20198100,1.286112,UP
2016-12-13,57.427836,58.273172,57.188938,57.868881,35718900,-0.478622,DOWN
2016-12-14,57.887258,58.300739,57.455399,57.593227,30352700,-0.159789,DOWN
...,...,...,...,...,...,...,...
2023-03-09,255.820007,259.559998,251.580002,252.320007,26653400,-1.500467,DOWN
2023-03-10,251.080002,252.789993,247.600006,248.589996,28321800,2.099087,UP
2023-03-13,247.399994,257.910004,245.729996,253.919998,33339700,2.634307,UP


# Simple Investment Strategy

### Create Strategy class

In [3]:
from backtesting import Strategy, Backtest

In [4]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [5]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [6]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting with specific values

In [7]:
model_dt.predict(df_explanatory)

array([0.08195771, 0.33014797, 0.08195771, ..., 0.31510252, 0.31510252,
       3.8957311 ])

In [8]:
results = bt.run()

### Interpret backtesting results

In [9]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2023-03-15 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],91.941624
Equity Final [$],54828.785476
Equity Peak [$],61047.729861
Return [%],448.287855


## Parametrize the Investment Strategy

### Create Strategy class

In [10]:
class Regression(Strategy):
    
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [11]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Optimize backtesting with multiple combinations

In [12]:
list_limits_buy = list(range(0, 11, 1))

In [13]:
list_limits_buy

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
list_limits_sell = list(range(0, -11, -1))

In [15]:
list_limits_sell

[0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10]

In [16]:
%%time

results = bt.optimize(
    limit_buy = list_limits_buy, limit_sell = list_limits_sell,
    maximize='Return [%]', return_heatmap=True
)

CPU times: user 3.23 s, sys: 28.9 ms, total: 3.26 s
Wall time: 2min 22s


### [ ] Interpret backtesting results

In [17]:
results_heatmap = results[1]

In [18]:
df_results_heatmap = results_heatmap.reset_index()

In [19]:
dff = df_results_heatmap.pivot(
    index='limit_buy', columns='limit_sell', values='Return [%]')

In [20]:
dff

limit_sell,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
limit_buy,,,,,,,,,,,
0,337.745113,337.745113,337.745113,352.472475,352.472475,422.353066,897.830150,2622.082816,3793.254167,11721.523635,10189.697691
1,282.772883,282.772883,282.772883,295.350207,295.350207,448.287855,872.765120,2933.971346,3885.646087,2529.445178,1143.188051
2,204.955270,204.955270,204.955270,215.076395,215.076395,236.764382,440.224724,579.616115,390.127920,-19.910033,-3.137514
3,204.955270,204.955270,204.955270,168.603790,168.603790,187.067913,212.102710,268.790120,138.430225,-45.846707,-31.507121
4,191.296430,191.296430,191.296430,155.713838,155.713838,241.940451,-42.654101,-72.062746,-74.879463,-76.810304,-69.170598
5,191.296430,191.296430,191.296430,155.713838,155.713838,87.385464,-78.964264,-100.000000,-100.000000,-100.000000,-100.000000
6,191.296430,191.296430,191.296430,155.713838,155.713838,-56.252997,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
7,191.296430,191.296430,191.296430,264.307150,264.307150,-40.761587,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
8,70.012364,70.012364,70.012364,70.012364,70.012364,-23.386490,-23.386490,-100.000000,-100.000000,-100.000000,-100.000000


In [21]:
dff.sort_index(axis=1, ascending=False)

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,10189.697691,11721.523635,3793.254167,2622.082816,897.830150,422.353066,352.472475,352.472475,337.745113,337.745113,337.745113
1,1143.188051,2529.445178,3885.646087,2933.971346,872.765120,448.287855,295.350207,295.350207,282.772883,282.772883,282.772883
2,-3.137514,-19.910033,390.127920,579.616115,440.224724,236.764382,215.076395,215.076395,204.955270,204.955270,204.955270
3,-31.507121,-45.846707,138.430225,268.790120,212.102710,187.067913,168.603790,168.603790,204.955270,204.955270,204.955270
4,-69.170598,-76.810304,-74.879463,-72.062746,-42.654101,241.940451,155.713838,155.713838,191.296430,191.296430,191.296430
5,-100.000000,-100.000000,-100.000000,-100.000000,-78.964264,87.385464,155.713838,155.713838,191.296430,191.296430,191.296430
6,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-56.252997,155.713838,155.713838,191.296430,191.296430,191.296430
7,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-40.761587,264.307150,264.307150,191.296430,191.296430,191.296430
8,-100.000000,-100.000000,-100.000000,-100.000000,-23.386490,-23.386490,70.012364,70.012364,70.012364,70.012364,70.012364


## DataFrame heatmaps for better reporting

In [22]:
dff.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient()

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,10190,11722,3793,2622,898,422,352,352,338,338,338
1,1143,2529,3886,2934,873,448,295,295,283,283,283
2,-3,-20,390,580,440,237,215,215,205,205,205
3,-32,-46,138,269,212,187,169,169,205,205,205
4,-69,-77,-75,-72,-43,242,156,156,191,191,191
5,-100,-100,-100,-100,-79,87,156,156,191,191,191
6,-100,-100,-100,-100,-100,-56,156,156,191,191,191
7,-100,-100,-100,-100,-100,-41,264,264,191,191,191
8,-100,-100,-100,-100,-23,-23,70,70,70,70,70


In [23]:
import numpy as np

In [24]:
dff.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient(vmin=np.nanmin(dff), vmax=np.nanmax(dff))

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,10190,11722,3793,2622,898,422,352,352,338,338,338
1,1143,2529,3886,2934,873,448,295,295,283,283,283
2,-3,-20,390,580,440,237,215,215,205,205,205
3,-32,-46,138,269,212,187,169,169,205,205,205
4,-69,-77,-75,-72,-43,242,156,156,191,191,191
5,-100,-100,-100,-100,-79,87,156,156,191,191,191
6,-100,-100,-100,-100,-100,-56,156,156,191,191,191
7,-100,-100,-100,-100,-100,-41,264,264,191,191,191
8,-100,-100,-100,-100,-23,-23,70,70,70,70,70
